In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [15]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [16]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.98


In [4]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [6]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [12]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

ValueError: labels ['smiles'] not contained in axis

In [ ]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [ ]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [17]:
RD = Ridge()
RD.fit(X_train, Y_train)
RD.score(X_train,Y_train)

0.46104094932887163

In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [ ]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)

In [4]:
n = np.arange(10, 150, 10) 
p = np.arange(200, 300, 10)
pd = pd.DataFrame(index = n, columns = p)
pd

,200,210,220,230,240,250,260,270,280,290
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
pd.set_value(20, 210, 50)
pd

,200,210,220,230,240,250,260,270,280,290
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import KFold
import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors

class MLObject():
    X = None
    INITIAL_FEATURE = 257
    FEAT = 'feat_'
    ALL_FEATURES_FUNCTIONS = [Descriptors.TPSA, Descriptors.MolWt]
    ALL_FEATURES_FUNCTIONS2 = [Chem.GetSSSR, Descriptors.TPSA, Descriptors.MolWt, Descriptors.ExactMolWt, 
                                Descriptors.NumValenceElectrons, Descriptors.NumRadicalElectrons, 
                                Descriptors.MaxPartialCharge, Descriptors.MinPartialCharge, Descriptors.MaxAbsPartialCharge, 
                                Descriptors.MinAbsPartialCharge, Descriptors.BalabanJ, Descriptors.BertzCT, Descriptors.Ipc,
                                Descriptors.HallKierAlpha, Descriptors.Kappa1, Descriptors.Kappa2, Descriptors.Kappa3,
                                Descriptors.Chi0, Descriptors.Chi1, Descriptors.Chi0n, Descriptors.Chi1n, Descriptors.Chi2n,
                                Descriptors.Chi3n, Descriptors.Chi4n, Descriptors.Chi0v, Descriptors.Chi1v, Descriptors.Chi2v,
                                Descriptors.Chi3v, Descriptors.Chi4v, Descriptors.MolLogP, Descriptors.MolMR, 
                                Descriptors.HeavyAtomCount, Descriptors.HeavyAtomMolWt, Descriptors.NHOHCount, 
                                Descriptors.NOCount, Descriptors.NumHAcceptors, Descriptors.NumHDonors, Descriptors.NumHeteroatoms,
                                Descriptors.NumRotatableBonds, Descriptors.RingCount, Descriptors.FractionCSP3,
                                Descriptors.LabuteASA,
                                Descriptors.NumAromaticRings, Descriptors.NumSaturatedRings, Descriptors.NumAliphaticRings]
    
    def __init__(self, file_name_, frac_=0, y_=True, drop_=None):
        self.file_name = file_name_
        self.all_df = pd.read_csv(self.file_name)
        if frac_ > 0:
            self.all_df = self.all_df.sample(frac=frac_)
        if y_ is True:
            self.y=self.all_df.ix[:, -1]
            self.X = self.all_df.ix[:, :-1]
        else:
            self.X = self.all_df
        if drop_ is not None:
            self.X = self.X.drop(drop_, axis=1)
        
    # def pre_process(self):

    def random_forest_tuning(self, min_estimators, max_estimators, min_features, max_features, k=10):
        # Parameters for tuning
        n_estimators = np.arange(min_estimators, max_estimators, 10)
        n_features = np.arange(min_features, max_features, 2)

        all_scores = pd.DataFrame(index=n_estimators,columns=n_features)

        for estimators in n_estimators:
            for features in n_features:
                model = RandomForestRegressor(n_estimators=estimators, max_features=features)
                score = self.run_model_kfold(model, k)

                # Update our record of the best parameters see so far
                all_scores.set_value(estimators, features, score)
        
        print(all_scores)

    def run_model_kfold(self, model, k):
        k_folds = KFold(self.X.shape[0],n_folds=k, shuffle=True)

        # want to add a validation set??
        scores = []
        for train_ind, test_ind in k_folds:
            # generate train data
            X_train = self.X.values[train_ind]
            y_train = self.y.values[train_ind]
            # generate test data
            X_test = self.X.values[test_ind]
            y_test = self.y.values[test_ind]

            model.fit(X_train, y_train)
            scores.append(model.score(X_test, y_test))
        return np.mean(scores)
    
    def time_start(self):
        self.start_time = time.time()
        
    def time_end(self):
        print("The function took ", time.time() - self.start_time, " seconds to run.")
    
    def calculate_features(self, s):
        mol = Chem.MolFromSmiles(s)
        return {'%s%s' % (FEAT, INITIAL_FEATURE + i): feature_function(mol) 
                for i, feature_function in enumerate(ALL_FEATURES_FUNCTIONS)}
    
    def pre_process(self): 
        df_list = np.split_array(self.X, 10000)
        for df in df_list:
            new_df = df['smiles'].apply(self.calculate_features)
            with open('output.csv') as f:
                new_df.to_csv(f, header=False)
            
        
        

/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/envs/rdkit/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [37]:
myobj = MLObject("train.csv", frac_ = 0.1)

In [38]:
newx = myobj.X[:100]

In [60]:
ALL_FEATURES_FUNCTIONS = [Chem.GetSSSR, Descriptors.TPSA, Descriptors.MolWt, Descriptors.ExactMolWt, 
                                Descriptors.NumValenceElectrons, Descriptors.NumRadicalElectrons, 
                                Descriptors.MaxPartialCharge, Descriptors.MinPartialCharge, Descriptors.MaxAbsPartialCharge, 
                                Descriptors.MinAbsPartialCharge, Descriptors.BalabanJ, Descriptors.BertzCT, Descriptors.Ipc,
                                Descriptors.HallKierAlpha, Descriptors.Kappa1, Descriptors.Kappa2, Descriptors.Kappa3,
                                Descriptors.Chi0, Descriptors.Chi1, Descriptors.Chi0n, Descriptors.Chi1n, Descriptors.Chi2n,
                                Descriptors.Chi3n, Descriptors.Chi4n, Descriptors.Chi0v, Descriptors.Chi1v, Descriptors.Chi2v,
                                Descriptors.Chi3v, Descriptors.Chi4v, Descriptors.MolLogP, Descriptors.MolMR, 
                                Descriptors.HeavyAtomCount, Descriptors.HeavyAtomMolWt, Descriptors.NHOHCount, 
                                Descriptors.NOCount, Descriptors.NumHAcceptors, Descriptors.NumHDonors, Descriptors.NumHeteroatoms,
                                Descriptors.NumRotatableBonds, Descriptors.RingCount, Descriptors.FractionCSP3,
                                Descriptors.LabuteASA,
                                Descriptors.NumAromaticRings, Descriptors.NumSaturatedRings, Descriptors.NumAliphaticRings]

INITIAL_FEATURE = 257
FEAT = 'feat_'

def calculate_features(s):
        mol = Chem.MolFromSmiles(s)
        return {'%s%s' % (FEAT, INITIAL_FEATURE + i): feature_function(mol) 
                for i, feature_function in enumerate(ALL_FEATURES_FUNCTIONS)}
    
def pre_process(X):
    col = 0; 
    df_list = np.array_split(X, 2)
    for df in df_list:
        result = pd.DataFrame(list(df['smiles'].apply(calculate_features)))
        # new_df = pd.concat([df, result], axis=1)
        with open("output.csv",'a') as f:
            if col == 0:
                result.to_csv(f, header=True, index=False)
                col += 1
            else: 
                result.to_csv(f, header=False, index=False)

In [61]:
pre_process(newx)

In [ ]:
        smile_strings = list(self.all_df['smiles'])
        mol_list = [Chem.MolFromSmiles(s) for s in smile_strings]
        
        df_all['feat_257'] = [Chem.GetSSSR(m) for m in mol_list]
        df_all['feat_258'] = [Descriptors.TPSA(m) for m in mol_list]
        df_all['feat_259'] = [Descriptors.MolWt(m) for m in mol_list]
        df_all['feat_260'] = [Descriptors.ExactMolWt(m) for m in mol_list]
        df_all['feat_261'] = [Descriptors.NumValenceElectrons(m) for m in mol_list]
        df_all['feat_262'] = [Descriptors.NumRadicalElectrons(m) for m in mol_list]
        df_all['feat_263'] = [Descriptors.MaxPartialCharge(m) for m in mol_list]
        df_all['feat_264'] = [Descriptors.MinPartialCharge(m) for m in mol_list]
        df_all['feat_265'] = [Descriptors.MaxAbsPartialCharge(m) for m in mol_list]
        df_all['feat_266'] = [Descriptors.MinAbsPartialCharge(m) for m in mol_list]
        df_all['feat_267'] = [Descriptors.BalabanJ(m) for m in mol_list]
        df_all['feat_268'] = [Descriptors.BertzCT(m) for m in mol_list]
        df_all['feat_269'] = [Descriptors.Ipc(m) for m in mol_list]
        df_all['feat_270'] = [Descriptors.HallKierAlpha(m) for m in mol_list]
        df_all['feat_271'] = [Descriptors.Kappa1(m) for m in mol_list]
        df_all['feat_272'] = [Descriptors.Kappa2(m) for m in mol_list]
        df_all['feat_273'] = [Descriptors.Kappa3(m) for m in mol_list]
        df_all['feat_274'] = [Descriptors.Chi0(m) for m in mol_list]
        df_all['feat_275'] = [Descriptors.Chi1(m) for m in mol_list]
        df_all['feat_276'] = [Descriptors.Chi0n(m) for m in mol_list]
        df_all['feat_277'] = [Descriptors.Chi1n(m) for m in mol_list]
        df_all['feat_278'] = [Descriptors.Chi2n(m) for m in mol_list]
        df_all['feat_279'] = [Descriptors.Chi3n(m) for m in mol_list]
        df_all['feat_280'] = [Descriptors.Chi4n(m) for m in mol_list]
        df_all['feat_281'] = [Descriptors.Chi0v(m) for m in mol_list]
        df_all['feat_282'] = [Descriptors.Chi1v(m) for m in mol_list]
        df_all['feat_283'] = [Descriptors.Chi2v(m) for m in mol_list]
        df_all['feat_284'] = [Descriptors.Chi3v(m) for m in mol_list]
        df_all['feat_285'] = [Descriptors.Chi4v(m) for m in mol_list]
        df_all['feat_286'] = [Descriptors.MolLogP(m) for m in mol_list]
        df_all['feat_287'] = [Descriptors.MolMR(m) for m in mol_list]
        df_all['feat_288'] = [Descriptors.HeavyAtomCount(m) for m in mol_list]
        df_all['feat_289'] = [Descriptors.HeavyAtomMolWt(m) for m in mol_list]
        df_all['feat_290'] = [Descriptors.NHOHCount(m) for m in mol_list]
        df_all['feat_291'] = [Descriptors.NOCount(m) for m in mol_list]
        df_all['feat_292'] = [Descriptors.NumHAcceptors(m) for m in mol_list]
        df_all['feat_293'] = [Descriptors.NumHDonors(m) for m in mol_list]
        df_all['feat_294'] = [Descriptors.NumHeteroatoms(m) for m in mol_list]
        df_all['feat_295'] = [Descriptors.NumRotatableBonds(m) for m in mol_list]
        df_all['feat_296'] = [Descriptors.NumAmideBonds(m) for m in mol_list]
        df_all['feat_297'] = [Descriptors.RingCount(m) for m in mol_list]
        df_all['feat_298'] = [Descriptors.FractionCSP3(m) for m in mol_list]
        df_all['feat_299'] = [Descriptors.NumSpiroAtoms(m) for m in mol_list]
        df_all['feat_300'] = [Descriptors.NumBridgeheadAtoms(m) for m in mol_list]
        df_all['feat_301'] = [Descriptors.LabuteASA(m) for m in mol_list]
        df_all['feat_302'] = [Descriptors.MQNs(m) for m in mol_list]
        df_all['feat_303'] = [Descriptors.NumAromaticRings(m) for m in mol_list]
        df_all['feat_304'] = [Descriptors.NumSaturatedRings(m) for m in mol_list]
        df_all['feat_305'] = [Descriptors.NumAliphaticRings(m) for m in mol_list]

In [ ]:
myobj = MLObject("train.csv", frac_ = 0.1, drop_ = ['smiles'])

In [8]:
myobj.random_forest_tuning(100, 120, 20, 40, 5)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/dorbaruch/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-a7e04cce686f>", line 1, in <module>
    myobj.random_forest_tuning(100, 120, 20, 40, 5)
  File "<ipython-input-6-8a4e98a7e093>", line 35, in random_forest_tuning
    score = self.run_model_kfold(model, k)
  File "<ipython-input-6-8a4e98a7e093>", line 55, in run_model_kfold
    model.fit(X_train, y_train)
  File "/Users/dorbaruch/anaconda/lib/python3.5/site-packages/sklearn/ensemble/forest.py", line 290, in fit
    for i, t in enumerate(trees))
  File "/Users/dorbaruch/anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py", line 800, in __call__
    while self.dispatch_one_batch(iterator):
  File "/Users/dorbaruch/anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py", line 658, in dispatch_one_batch
    self._dispatc

KeyboardInterrupt: 

In [ ]:
df_feat = pd.read_csv(f)

for column in df_deaf.columns:
    df_deaf[column].fillna((df_deaf[column].mean()), inplace=True)

In [19]:
def ridge_tuning(min_alpha, max_alpha, k=10):
    # Parameters for tuning
    alpha_tune = min_alpha

    all_scores = pd.DataFrame(index=[1],columns=n_features)
    
    while alpha < max_alpha:
        model = Ridge(alpha=alpha_tune)
        score = self.run_model_kfold(model, k)
        alpha_tune = alpha_tune * 10
        
        # Update our record of the best parameters see so far
        all_scores.set_value(1, features, score)

    print(all_scores)

def random_forest_tuning(min_features, max_features, k=10):
    # Parameters for tuning
    n_features = np.arange(min_features, max_features, 2)

    all_scores = pd.DataFrame(index=[1],columns=n_features)
    
    for features in n_features:
        model = RandomForestRegressor(n_estimators=estimators, max_features=features)
        score = self.run_model_kfold(model, k)

        # Update our record of the best parameters see so far
        all_scores.set_value(1, features, score)

    print(all_scores)

def run_model_kfold(self, model, k):
    k_folds = KFold(self.X.shape[0],n_folds=k, shuffle=True)

    # want to add a validation set??
    scores = []
    for train_ind, test_ind in k_folds:
        # generate train data
        X_train = self.X.values[train_ind]
        y_train = self.y.values[train_ind]
        # generate test data
        X_test = self.X.values[test_ind]
        y_test = self.y.values[test_ind]

        model.fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
    return np.mean(scores)

def time_start(self):
    self.start_time = time.time()

def time_end(self):
    print("The function took ", time.time() - self.start_time, " seconds to run.")